In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model,cluster
from sklearn.metrics import mean_squared_error, r2_score 
import pandas as pd
import sklearn.model_selection
from collections import defaultdict, Counter
import math

In [2]:
df = pd.read_csv("zip_housing.csv")

In [3]:
df = df.dropna(subset=['lot_size','building_size'])

In [4]:
df

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code
0,O2889402305,multi_family,1906.0,f,NaN,NaN,NaN,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
2,O9299703972,multi_family,1938.0,f,NaN,NaN,NaN,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001
3,O2866856410,single_family,1905.0,f,4.0,2.0,NaN,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001
4,O2823776981,condo,1912.0,f,5.0,2.0,NaN,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98072,O1311909799,single_family,NaN,f,NaN,NaN,NaN,NaN,78000,10454.0,0.0,973 Albion Ave,"['North Redding', 'Land Park', 'Quartz Hill']",Redding,Shasta,CA,96003
98073,O1392454616,single_family,2006.0,f,3.0,2.0,1.0,NaN,275000,2614.0,1430.0,2365 LA Villa Way,"['Northeast Redding', 'The Villages at Shasta ...",Redding,Shasta,CA,96003
98075,O1290952370,single_family,1977.0,f,3.0,2.0,NaN,5.0,483500,89734.0,2552.0,19287 Roxie Ln,NaN,Redding,Shasta,CA,96003
98076,O1523744489,single_family,1984.0,f,4.0,3.0,NaN,4.0,345000,87120.0,2040.0,13143 Tamera Way,NaN,Redding,Shasta,CA,96003


In [5]:
df[np.abs(df.price-df.price.mean())<=(3*df.price.std())]

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code
0,O2889402305,multi_family,1906.0,f,NaN,NaN,NaN,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
2,O9299703972,multi_family,1938.0,f,NaN,NaN,NaN,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001
3,O2866856410,single_family,1905.0,f,4.0,2.0,NaN,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001
4,O2823776981,condo,1912.0,f,5.0,2.0,NaN,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98072,O1311909799,single_family,NaN,f,NaN,NaN,NaN,NaN,78000,10454.0,0.0,973 Albion Ave,"['North Redding', 'Land Park', 'Quartz Hill']",Redding,Shasta,CA,96003
98073,O1392454616,single_family,2006.0,f,3.0,2.0,1.0,NaN,275000,2614.0,1430.0,2365 LA Villa Way,"['Northeast Redding', 'The Villages at Shasta ...",Redding,Shasta,CA,96003
98075,O1290952370,single_family,1977.0,f,3.0,2.0,NaN,5.0,483500,89734.0,2552.0,19287 Roxie Ln,NaN,Redding,Shasta,CA,96003
98076,O1523744489,single_family,1984.0,f,4.0,3.0,NaN,4.0,345000,87120.0,2040.0,13143 Tamera Way,NaN,Redding,Shasta,CA,96003


In [6]:
temp = sorted(df[np.abs(df.price-df.price.mean())<=(3*df.price.std())]['price'])
for i in temp:
    print(i)

909
3000
4500
4900
5000
6000
6000
6000
6500
10000
10000
10000
10000
10000
10500
14000
14000
14000
14000
14000
14000
15000
15000
15000
15000
15000
15000
15000
16000
16000
18000
18000
20000
20000
20000
20000
20000
20000
20000
20500
20500
22500
22500
22500
22907
22907
22907
22907
22907
22907
22907
22907
22907
22907
22907
22907
22907
22907
22907
24000
24000
24000
25000
25000
25000
25000
25000
25424
25500
26000
26500
27000
27500
27500
28000
28000
28000
28000
28000
28000
29500
30000
30000
30000
30000
31000
32000
32000
34000
35000
35000
35000
35000
35000
35000
35000
36000
37000
37500
37500
37500
39000
39900
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
42000
42000
45000
45000
45000
45000
45000
45000
45000
45000
45000
45000
45000
45000
45000
45000
45500
46000
46500
47000
47000
47000
47000
47500
47900
48000
48000
48000
48000
48500
48777
49000
49000
49500
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50

199000
199000
199000
199000
199000
199500
199500
199500
199500
199900
199900
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000
200000

255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255000
255050
255182
255500
255500
255500
255500
255570
255750
255888
256000
256000
256000
256000
256000
256000
256000
256000
256000
256000
256000
256000
256000
256000
256000
256500
256500
256500
256500
256500
256873
257000
257000
257000
257000
257000
257000
257000
257000
257000
257000
257000
257000
257000
257000
257000
257500
257500
257500
257500
257500
257500
257500
257500
257500
257500
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258000
258200
258500
258500
258500
258831
259000
259000
259000
259000
259000
259000
259000
259000
259000
259000
259000
259000

295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295000
295101
295222
295500
295500
295500
295500
295500
295500
295500
295900
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296000
296200
296500
296500
296500
296500
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297000
297500
297500
297500
297500
297500
297500
297500
297500
297500
297500
297800
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298000
298500
298500
298500
298500
298500
298500
298700
298888
299000
299000
299000
299000
299000
299000
299000

325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000
325000

...

1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480000
1480088
1481000
1481000
1481000
1481000
1481500
1481562
1482000
1482000
1482000
1482000
1482000
1482000
1482320
1482500
1482925
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1485000
1486510
1486753
1487000
1487000
1487000
1487000
1487000
1487500
1488000
1488000
1488000
1488000
1488000
1488000
1488000
1488888
1489000
1489000
1489000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1490000
1491000
1492500


1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675000
1675180
1676000
1677000
1677273
1677301
1678000
1678000
1678000
1678750
1679000
1679000
1679500
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680000
1680500
1680600
1681000
1682000
1682000
1682500
1684000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685000
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1685500
1686837
1687000
1688000
1688000
1688000
1688000
1688800
1688911
1689000
1690000
1690000
1690000
1690000
1690000


1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1995000
1998000
1998000
1998000
1998000
1998000
1998001
1998800
1998888
1999000
1999000
1999000
1999000
1999000
1999000
1999500
1999988
1999990
1999999
1999999
1999999
1999999
1999999
1999999
1999999
1999999
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000


2625000
2625000
2625000
2625000
2625000
2626500
2628000
2630000
2630000
2630000
2630000
2630000
2630000
2630000
2634000
2634000
2634000
2634000
2635000
2635000
2635000
2640000
2640000
2640000
2640000
2645000
2645000
2645000
2645000
2645000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2650000
2651000
2652000
2655000
2655000
2655000
2655000
2657850
2658000
2658000
2660000
2660000
2660000
2660000
2660000
2660000
2665000
2665000
2665000
2667000
2668000
2668000
2670000
2670000
2670000
2670000
2670000
2670000
2670000
2673750
2675000
2675000
2675000
2675000
2675000
2675000
2675000
2675000
2675000
2675000
2675000
2675000
2680000
2680000
2680000
2680000
2680000
2680000


8479000
8482000
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8493500
8500000
8500000
8521000
8600000
8600000
8600000
8620000
8640000
8650000
8743000
8750000
8750000
8800000
8800000
8800000
8814000
8814000
8814000
8814000
8814000
8814000
8825000
8875000
8900000
8900000
8900000
8900000
8920000
9000000
9050000
9056000
9200000
9300000
9375000
9375000
9400000
9405000
9450000
9450000
9495000
9500000
9500000
9500000
9500000
9590000
9652500
9667000
9750000
9800000
9806000
9850000
9850000
9875000
9900000
9950000
9995000
10000000
10000000
10000000
10000000
10000000
10000000
10000000
10000000
10100000
10160000
10208000
10250000
10259688
10300000
10350000
10374500
10375000
10395000
10400000
10400000
10400000
10400000
10481500
10481500


In [7]:
df[np.abs(df.lot_size-df.lot_size.mean())<=(3*df.lot_size.std())]

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code
0,O2889402305,multi_family,1906.0,f,NaN,NaN,NaN,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
2,O9299703972,multi_family,1938.0,f,NaN,NaN,NaN,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001
3,O2866856410,single_family,1905.0,f,4.0,2.0,NaN,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001
4,O2823776981,condo,1912.0,f,5.0,2.0,NaN,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98072,O1311909799,single_family,NaN,f,NaN,NaN,NaN,NaN,78000,10454.0,0.0,973 Albion Ave,"['North Redding', 'Land Park', 'Quartz Hill']",Redding,Shasta,CA,96003
98073,O1392454616,single_family,2006.0,f,3.0,2.0,1.0,NaN,275000,2614.0,1430.0,2365 LA Villa Way,"['Northeast Redding', 'The Villages at Shasta ...",Redding,Shasta,CA,96003
98075,O1290952370,single_family,1977.0,f,3.0,2.0,NaN,5.0,483500,89734.0,2552.0,19287 Roxie Ln,NaN,Redding,Shasta,CA,96003
98076,O1523744489,single_family,1984.0,f,4.0,3.0,NaN,4.0,345000,87120.0,2040.0,13143 Tamera Way,NaN,Redding,Shasta,CA,96003


In [8]:
df[np.abs(df.building_size-df.building_size.mean())<=(3*df.building_size.std())]

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code
0,O2889402305,multi_family,1906.0,f,NaN,NaN,NaN,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001
2,O9299703972,multi_family,1938.0,f,NaN,NaN,NaN,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001
3,O2866856410,single_family,1905.0,f,4.0,2.0,NaN,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001
4,O2823776981,condo,1912.0,f,5.0,2.0,NaN,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98072,O1311909799,single_family,NaN,f,NaN,NaN,NaN,NaN,78000,10454.0,0.0,973 Albion Ave,"['North Redding', 'Land Park', 'Quartz Hill']",Redding,Shasta,CA,96003
98073,O1392454616,single_family,2006.0,f,3.0,2.0,1.0,NaN,275000,2614.0,1430.0,2365 LA Villa Way,"['Northeast Redding', 'The Villages at Shasta ...",Redding,Shasta,CA,96003
98075,O1290952370,single_family,1977.0,f,3.0,2.0,NaN,5.0,483500,89734.0,2552.0,19287 Roxie Ln,NaN,Redding,Shasta,CA,96003
98076,O1523744489,single_family,1984.0,f,4.0,3.0,NaN,4.0,345000,87120.0,2040.0,13143 Tamera Way,NaN,Redding,Shasta,CA,96003


In [9]:
db = cluster.DBSCAN(eps=0.5,min_samples=5, metric='euclidean', 
                    metric_params=None, algorithm='auto', 
                    leaf_size=30, p=None, n_jobs=None)

In [10]:
model = db.fit(df[['price']])

In [11]:
temp = []
for i in model.labels_:
    if i == -1:
        temp.append(1)
    else:
        temp.append(0)
print(len(temp))

85291


In [12]:
df['outliers'] = temp

In [13]:
df.sort_values(by='price', ascending=False)

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code,outliers
24990,O2155717278,single_family,2018.0,f,4.0,3.0,1.0,3.0,915000000,30492.0,2575.0,3146 Roadrunner Rd,['Business District'],San Marcos,San Diego,CA,92078,1
22360,O2675590810,single_family,1985.0,f,4.0,3.0,NaN,3.0,865000000,6746.0,1980.0,2329 Wind River Rd,NaN,El Cajon,San Diego,CA,92019,1
27861,O2063411801,multi_family,1940.0,f,3.0,2.0,NaN,NaN,515000000,3437.0,1336.0,2916-2918 Logan Ave,"['Central San Diego', 'Logan Heights']",San Diego,San Diego,CA,92113,1
78506,O9250147220,commercial,2018.0,f,NaN,NaN,NaN,NaN,275000000,113025.0,0.0,1149 Coleman Ave,NaN,San Jose,Santa Clara,CA,95110,1
65703,O9754401764,single_family,NaN,f,NaN,NaN,NaN,NaN,202000000,31400.0,0.0,300 Channel St,"['South of Market', 'Design District', 'Showpl...",San Francisco,San Francisco,CA,94158,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24785,O9891051440,land,NaN,f,NaN,NaN,NaN,NaN,5000,3920.0,0.0,162 Johnston Ln,['Country Club Heights Lakeland'],San Marcos,Riverside,CA,92069,1
73156,O9620529933,land,NaN,f,NaN,NaN,NaN,NaN,4900,45738.0,0.0,3939 S 6th St,NaN,Vallejo,Klamath,CA,94590,1
52667,O2714403057,NaN,NaN,f,NaN,NaN,NaN,NaN,4500,7228.0,0.0,1215 Truxtun Ave,NaN,Bakersfield,Kern,CA,93301,1
22728,O1316107888,land,NaN,f,NaN,NaN,NaN,NaN,3000,13383.0,0.0,168 Marsden Ct,NaN,El Cajon,Imperial,CA,92020,1


In [14]:
df.head(30)

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code,outliers
0,O2889402305,multi_family,1906.0,f,NaN,NaN,NaN,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001,0
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001,0
2,O9299703972,multi_family,1938.0,f,NaN,NaN,NaN,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001,0
3,O2866856410,single_family,1905.0,f,4.0,2.0,NaN,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001,0
4,O2823776981,condo,1912.0,f,5.0,2.0,NaN,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001,0
5,O1174561323,single_family,1920.0,f,5.0,5.0,NaN,NaN,477000,6043.0,2344.0,7537 Maie Ave,['South LA'],Los Angeles,Los Angeles,CA,90001,0
6,O1387647983,multi_family,1971.0,f,8.0,4.0,NaN,NaN,600000,4365.0,2616.0,8308 Hooper Ave,['South LA'],Los Angeles,Los Angeles,CA,90001,0
7,O1238994138,single_family,1930.0,f,2.0,1.0,NaN,1.0,420000,1536.0,784.0,7708 Stanford Ave,"['South LA', 'Florence']",Los Angeles,Los Angeles,CA,90001,0
8,O1296756349,multi_family,1964.0,f,NaN,NaN,NaN,3.0,545000,6000.0,1568.0,8126 Crockett Blvd,['South LA'],Los Angeles,Los Angeles,CA,90001,0
9,O1223156112,single_family,1937.0,f,3.0,1.0,1.0,2.0,530000,5103.0,1198.0,649 E 76th Pl,"['South LA', 'Florence']",Los Angeles,Los Angeles,CA,90001,0


In [15]:
housing = df[df['outliers'] == 0]

In [16]:
housing

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code,outliers
0,O2889402305,multi_family,1906.0,f,NaN,NaN,NaN,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001,0
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001,0
2,O9299703972,multi_family,1938.0,f,NaN,NaN,NaN,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001,0
3,O2866856410,single_family,1905.0,f,4.0,2.0,NaN,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001,0
4,O2823776981,condo,1912.0,f,5.0,2.0,NaN,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98071,O2868664313,single_family,NaN,f,NaN,NaN,NaN,NaN,370000,10890.0,0.0,3523 Americana Way,"['North Redding', 'Quartz Hill', 'River Ridge ...",Redding,Shasta,CA,96003,0
98073,O1392454616,single_family,2006.0,f,3.0,2.0,1.0,NaN,275000,2614.0,1430.0,2365 LA Villa Way,"['Northeast Redding', 'The Villages at Shasta ...",Redding,Shasta,CA,96003,0
98075,O1290952370,single_family,1977.0,f,3.0,2.0,NaN,5.0,483500,89734.0,2552.0,19287 Roxie Ln,NaN,Redding,Shasta,CA,96003,0
98076,O1523744489,single_family,1984.0,f,4.0,3.0,NaN,4.0,345000,87120.0,2040.0,13143 Tamera Way,NaN,Redding,Shasta,CA,96003,0


In [17]:
housing[['baths_half']] = housing[['baths_half']].fillna(0)
housing

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,postal_code,outliers
0,O2889402305,multi_family,1906.0,f,NaN,NaN,0.0,3.0,570000,7057.0,1756.0,6375 Converse Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001,0
1,O1305476579,single_family,1913.0,f,3.0,1.0,1.0,1.0,470000,5721.0,1030.0,7916 Alix Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,90001,0
2,O9299703972,multi_family,1938.0,f,NaN,NaN,0.0,NaN,830000,5495.0,2503.0,1222 E 78th St,['South LA'],Los Angeles,Los Angeles,CA,90001,0
3,O2866856410,single_family,1905.0,f,4.0,2.0,0.0,NaN,400000,5387.0,1248.0,1659 E 70th St,['South LA'],Los Angeles,Los Angeles,CA,90001,0
4,O2823776981,condo,1912.0,f,5.0,2.0,0.0,NaN,659000,5667.0,1897.0,1364 E 58th Pl,['South LA'],Los Angeles,Los Angeles,CA,90001,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98071,O2868664313,single_family,NaN,f,NaN,NaN,0.0,NaN,370000,10890.0,0.0,3523 Americana Way,"['North Redding', 'Quartz Hill', 'River Ridge ...",Redding,Shasta,CA,96003,0
98073,O1392454616,single_family,2006.0,f,3.0,2.0,1.0,NaN,275000,2614.0,1430.0,2365 LA Villa Way,"['Northeast Redding', 'The Villages at Shasta ...",Redding,Shasta,CA,96003,0
98075,O1290952370,single_family,1977.0,f,3.0,2.0,0.0,5.0,483500,89734.0,2552.0,19287 Roxie Ln,NaN,Redding,Shasta,CA,96003,0
98076,O1523744489,single_family,1984.0,f,4.0,3.0,0.0,4.0,345000,87120.0,2040.0,13143 Tamera Way,NaN,Redding,Shasta,CA,96003,0


In [18]:
housing_grouped = housing.groupby('beds')
count_1 = 0
housing_after_fill_baths = pd.DataFrame()
for name, group in housing_grouped:
    ##print(len(group))
    group[["baths_full"]].fillna(group.median(), inplace = True)
    group
    ##group[["baths_full"]] = group[["baths_full"]].astype({"baths_full": int})
    ##group.fillna(group.median(), inplace = True)
    ##print(group)
    ##truncate_num = int(len(group)*0.05)
    ##truncate_len = len(group) - truncate_num
    ##group.sort_values(by=['price'])
    ##group.truncate(before = truncate_num, after = len(group) - truncate_num)
    ##group = group.head(truncate_len)
    ##truncate_len -= truncate_num
    ##group = group.tail(truncate_len)
    ##group = group.fillna(group.median(), inplace = True)
    ##group
    if(count_1 == 0):
        housing_after_fill_baths = group
        count_1 += 1
    else:
        housing_after_fill_baths = pd.concat([housing_after_fill_baths, group])
housing_after_fill_baths
housing_after_fill_baths[['baths_full']] = housing_after_fill_baths[["baths_full"]].fillna(housing_after_fill_baths.median()).astype({"baths_full": int})
housing_after_fill_baths[["baths_full"]] = housing_after_fill_baths[["baths_full"]].astype({"baths_full": int})
housing_after_fill_baths['baths'] = housing_after_fill_baths['baths_full'] + housing_after_fill_baths['baths_half']*0.5
housing_after_fill_baths.set_index('postal_code', inplace = True)
housing_after_fill_baths

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,property_id,prop_type,year_built,is_new_construction,beds,baths_full,baths_half,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,outliers,baths
postal_code,,,,,,,,,,,,,,,,,,
90006,O2162213627,condo,1923.0,f,0.0,2,0.0,NaN,290000,21307.0,460.0,2121 James M Wood Blvd Apt 420,"['Westlake', 'Central LA', 'Macarthur Park']",Los Angeles,Los Angeles,CA,0,2.0
90006,O1856851419,condo,1923.0,f,0.0,1,0.0,NaN,265000,21307.0,450.0,2121 James M Wood Blvd Apt 315,"['MacArthur Park', 'Central LA']",Los Angeles,Los Angeles,CA,0,1.0
90011,O2755958040,single_family,2020.0,t,0.0,2,0.0,NaN,1450000,17205.0,17205.0,824 E Jefferson Blvd,"['South LA', 'South Central LA']",Los Angeles,Los Angeles,CA,0,2.0
90012,O1470967874,condo,1910.0,f,0.0,1,0.0,NaN,680000,19153.0,1050.0,108 W 2nd St Apt 413,"['Central LA', 'Gallery Row', 'Downtown Los An...",Los Angeles,Los Angeles,CA,0,1.0
90012,O2118639986,condo,1910.0,f,0.0,1,0.0,NaN,449000,19153.0,799.0,108 W 2nd St Apt 510,"['Central LA', 'Gallery Row', 'Downtown Los An...",Los Angeles,Los Angeles,CA,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93703,O2602925890,multi_family,NaN,f,60.0,40,0.0,NaN,375000,8400.0,2966.0,2136 N 1st St,['McLane'],Fresno,Fresno,CA,0,40.0
94062,O2551374403,multi_family,1967.0,f,84.0,8,0.0,NaN,306500,49658.0,0.0,235 Arlington Rd,"['Wellesley Park', 'Edgewood Park']",Redwood City,San Mateo,CA,0,8.0
90059,O1325347458,multi_family,1970.0,f,88.0,44,0.0,NaN,10500000,43521.0,28824.0,937 E 120th St,"['South LA', 'Harbor Gateway North']",Los Angeles,Los Angeles,CA,0,44.0


In [19]:
##housing_after_fill_baths.shape()
housing_grouped = housing_after_fill_baths.groupby(['postal_code', 'baths'])
count = 0
housing_after_all_nan = pd.DataFrame()
for name, group in housing_grouped:
    ##print(len(group))
    group.fillna(group.median(), inplace = True)
    ##print(group)
    ##truncate_num = int(len(group)*0.05)
    ##truncate_len = len(group) - truncate_num
    ##group.sort_values(by=['price'])
    ##group.truncate(before = truncate_num, after = len(group) - truncate_num)
    ##group = group.head(truncate_len)
    ##truncate_len -= truncate_num
    ##group = group.tail(truncate_len)
    ##group = group.fillna(group.median(), inplace = True)
    ##group
    if(count == 0):
        housing_after_all_nan = group
        count += 1
    else:
        housing_after_all_nan = pd.concat([housing_after_all_nan, group])

In [20]:
housing_after_all_nan = housing_after_all_nan[['property_id', 'prop_type', 'year_built', 'is_new_construction', 'beds', 'baths', 'garage'
        , 'price', 'lot_size', 'building_size', 'line', 'neighborhoods', 'city', 'county', 'state_code'
         , 'baths_full', 'baths_half']]

In [21]:
housing_after_all_nan.dropna(subset=['year_built', 'is_new_construction', 'beds', 'baths',
                                    'garage', 'price', 'lot_size', 'building_size'], inplace = True)
housing_after_all_nan

,property_id,prop_type,year_built,is_new_construction,beds,baths,garage,price,lot_size,building_size,line,neighborhoods,city,county,state_code,baths_full,baths_half
postal_code,,,,,,,,,,,,,,,,,
90001,O1375275690,single_family,1928.0,f,1.0,1.0,1.0,216000,2813.0,540.0,8322 Hickory St,['South LA'],Los Angeles,Los Angeles,CA,1,0.0
90001,O1411833579,single_family,1929.0,f,1.0,1.0,1.0,280000,1700.0,537.0,701 E 84th St,"['South LA', 'Florence']",Los Angeles,Los Angeles,CA,1,0.0
90001,O1238994138,single_family,1930.0,f,2.0,1.0,1.0,420000,1536.0,784.0,7708 Stanford Ave,"['South LA', 'Florence']",Los Angeles,Los Angeles,CA,1,0.0
90001,O2786660102,single_family,1923.0,f,2.0,1.0,1.0,421000,5597.0,932.0,1230 E 59th Pl,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,1,0.0
90001,O1302346240,single_family,1925.0,f,2.0,1.0,1.0,350000,6604.0,853.0,7919 Morton Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles,CA,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96003,O2050994416,single_family,1992.0,f,3.0,3.5,3.0,410000,16553.0,2289.0,6833 Terra Linda Way,"['North Redding', 'Country Oaks', 'Midway']",Redding,Shasta,CA,3,1.0
96003,O1045697006,single_family,1979.0,f,5.0,4.0,2.0,640000,296208.0,3348.0,20114 Vernita Dr,NaN,Redding,Shasta,CA,4,0.0
96003,O1528153242,multi_family,1978.0,f,6.0,4.0,3.5,6500000,14375.0,2895.0,285 Boulder Creek Dr,"['Twinview', 'North Redding']",Redding,Shasta,CA,4,0.0


In [22]:
housing_after_all_nan.insert (3, 'is_multi_family', np.where(housing_after_all_nan['prop_type'] == 'multi_family', 1, 0))
housing_after_all_nan.insert (4, 'is_single_family', np.where(housing_after_all_nan['prop_type'] == 'single_family', 1, 0))
housing_after_all_nan.insert (5, 'is_condo', np.where(housing_after_all_nan['prop_type'] == 'condo', 1, 0))
housing_after_all_nan.drop(columns=['prop_type', 'state_code', 'property_id', 'baths_full', 'baths_half'], inplace = True)
housing_after_all_nan.loc[(housing_after_all_nan.is_new_construction == 'f'),'is_new_construction']= 0
housing_after_all_nan.loc[(housing_after_all_nan.is_new_construction != False),'is_new_construction']= 1

In [23]:
housing_after_all_nan["baths_beds_ratio"] = housing_after_all_nan['beds'].where(housing_after_all_nan['beds'] == 0, round(housing_after_all_nan['baths']/housing_after_all_nan['beds'], 4))
housing_after_all_nan.insert(7, "baths_beds_ratio", housing_after_all_nan.pop("baths_beds_ratio"))

In [24]:
housing_after_all_nan

,year_built,is_multi_family,is_single_family,is_condo,is_new_construction,beds,baths,baths_beds_ratio,garage,price,lot_size,building_size,line,neighborhoods,city,county
postal_code,,,,,,,,,,,,,,,,
90001,1928.0,0,1,0,0,1.0,1.0,1.0000,1.0,216000,2813.0,540.0,8322 Hickory St,['South LA'],Los Angeles,Los Angeles
90001,1929.0,0,1,0,0,1.0,1.0,1.0000,1.0,280000,1700.0,537.0,701 E 84th St,"['South LA', 'Florence']",Los Angeles,Los Angeles
90001,1930.0,0,1,0,0,2.0,1.0,0.5000,1.0,420000,1536.0,784.0,7708 Stanford Ave,"['South LA', 'Florence']",Los Angeles,Los Angeles
90001,1923.0,0,1,0,0,2.0,1.0,0.5000,1.0,421000,5597.0,932.0,1230 E 59th Pl,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles
90001,1925.0,0,1,0,0,2.0,1.0,0.5000,1.0,350000,6604.0,853.0,7919 Morton Ave,"['Florence-Graham', 'South LA']",Los Angeles,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96003,1992.0,0,1,0,0,3.0,3.5,1.1667,3.0,410000,16553.0,2289.0,6833 Terra Linda Way,"['North Redding', 'Country Oaks', 'Midway']",Redding,Shasta
96003,1979.0,0,1,0,0,5.0,4.0,0.8000,2.0,640000,296208.0,3348.0,20114 Vernita Dr,NaN,Redding,Shasta
96003,1978.0,1,0,0,0,6.0,4.0,0.6667,3.5,6500000,14375.0,2895.0,285 Boulder Creek Dr,"['Twinview', 'North Redding']",Redding,Shasta


In [26]:
housing_after_all_nan.to_csv("output.csv")